In [1]:
#Importing libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
#First scraping one season of data
url = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'

In [3]:
#Collecting html data 
html = urlopen(url)

#Creating beautiful soup object
soup = BeautifulSoup(html, features='lxml')

In [8]:
print(soup)

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202308191" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLo

In [19]:
#Extracting headers to a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

In [26]:
print(headers)

['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


In [25]:
#Removing Rk so that the column dimensions match
headers.remove('Rk')

In [22]:
#Obtaining rows from the table
rows = soup.findAll('tr')[1:]
rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

In [23]:
print(rows_data)

[['Precious Achiuwa', 'C', '23', 'TOR', '55', '12', '20.7', '3.6', '7.3', '.485', '0.5', '2.0', '.269', '3.0', '5.4', '.564', '.521', '1.6', '2.3', '.702', '1.8', '4.1', '6.0', '0.9', '0.6', '0.5', '1.1', '1.9', '9.2'], ['Steven Adams', 'C', '29', 'MEM', '42', '42', '27.0', '3.7', '6.3', '.597', '0.0', '0.0', '.000', '3.7', '6.2', '.599', '.597', '1.1', '3.1', '.364', '5.1', '6.5', '11.5', '2.3', '0.9', '1.1', '1.9', '2.3', '8.6'], ['Bam Adebayo', 'C', '25', 'MIA', '75', '75', '34.6', '8.0', '14.9', '.540', '0.0', '0.2', '.083', '8.0', '14.7', '.545', '.541', '4.3', '5.4', '.806', '2.5', '6.7', '9.2', '3.2', '1.2', '0.8', '2.5', '2.8', '20.4'], ['Ochai Agbaji', 'SG', '22', 'UTA', '59', '22', '20.5', '2.8', '6.5', '.427', '1.4', '3.9', '.355', '1.4', '2.7', '.532', '.532', '0.9', '1.2', '.812', '0.7', '1.3', '2.1', '1.1', '0.3', '0.3', '0.7', '1.7', '7.9'], ['Santi Aldama', 'PF', '22', 'MEM', '77', '20', '21.8', '3.2', '6.8', '.470', '1.2', '3.5', '.353', '2.0', '3.4', '.591', '.560', '

In [29]:
#Creating a dataframe and exporting to csv
nbaStats23 = pd.DataFrame(rows_data, columns = headers)
nbaStats23.to_csv(r'C:\Users\ryant\OneDrive\Documents\nbaStats23.csv', index=False)

In [28]:
nbaStats23.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,.485,...,.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,.597,...,.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,.540,...,.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,.427,...,.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,.470,...,.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0


In [47]:
#Function to scrape multiple seasons of data
def scraper(years):
    final_df = pd.DataFrame(columns = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
                                       '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
                                       'STL', 'BLK', 'TOV', 'PF', 'PTS'])
    #loop through years
    for y in years:
        #season to scrape
        year = y
        #url of interest to scrape
        url = f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'
        #fixing too many requests issue
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
        requests.get(url, headers=headers)
        #collecting html data
        html = urlopen(url)
        #creating BeautifuSoup object
        soup = BeautifulSoup(html, features="lxml")
        #using getText() to extract headers
        titles = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
        print(titles)
        #find only column headers
        headers1 = titles[1:titles.index("PF")+1]
        #now extract all data from rows
        rows = soup.findAll('tr')[1:]
        stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        #only keep the required rows
        stats = [e for e in stats if e != []]
        #adding the team name to each row in team_stats
        for i in range(0, len(stats)):
            #stats[i].insert(0, row_titles[i])
            stats[i].insert(0, year)
        #adding year to columns
        headers1.insert(0, "Year")
        #creating a dataframe with all the info
        year_stats = pd.DataFrame(stats, columns = titles)
        #append this new dataframe to final_df
        final_df = final_df.append(year_stats)
    #print final_df
    print(final_df.info)
    #export to csv
    final_df.to_csv("nba_team_data.csv", index=False)
    return final_df

In [31]:
seasons = [2018, 2019, 2020, 2021, 2022, 2023]

In [48]:
df = scraper(seasons)

['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


C:\Users\ryant\AppData\Local\Temp\ipykernel_12668\3640709299.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(year_stats)


['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


C:\Users\ryant\AppData\Local\Temp\ipykernel_12668\3640709299.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(year_stats)


['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


...

C:\Users\ryant\AppData\Local\Temp\ipykernel_12668\3640709299.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(year_stats)


['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


C:\Users\ryant\AppData\Local\Temp\ipykernel_12668\3640709299.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(year_stats)


['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
<bound method DataFrame.info of              Player Pos Age   Tm   G  GS    MP   FG   FGA   FG%  ...  ORB  \
0      Álex Abrines  SG  24  OKC  75   8  15.1  1.5   3.9  .395  ...  0.3   
1        Quincy Acy  PF  27  BRK  70   8  19.4  1.9   5.2  .356  ...  0.6   
2      Steven Adams   C  24  OKC  76  76  32.7  5.9   9.4  .629  ...  5.1   
3       Bam Adebayo   C  20  MIA  69  19  19.8  2.5   4.9  .512  ...  1.7   
4     Arron Afflalo  SG  32  ORL  53   3  12.9  1.2   3.1  .401  ...  0.1   
..              ...  ..  ..  ...  ..  ..   ...  ...   ...   ...  ...  ...   
674  Thaddeus Young  PF  34  TOR  54   9  14.7  2.0   3.7  .545  ...  1.3   
675      Trae Young  PG  24  ATL  73  73  34.8  8.2  19.0  .429  ...  0.8   
676  Omer Yurtseven   C  24  MIA   9   0   9.2  1.8   3.0  .593  ...  0

C:\Users\ryant\AppData\Local\Temp\ipykernel_12668\3640709299.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(year_stats)


In [53]:
df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Álex Abrines,SG,24,OKC,75,8,15.1,1.5,3.9,.395,...,0.3,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7,2018
1,Quincy Acy,PF,27,BRK,70,8,19.4,1.9,5.2,.356,...,0.6,3.1,3.7,0.8,0.5,0.4,0.9,2.1,5.9,2018
2,Steven Adams,C,24,OKC,76,76,32.7,5.9,9.4,.629,...,5.1,4.0,9.0,1.2,1.2,1.0,1.7,2.8,13.9,2018
3,Bam Adebayo,C,20,MIA,69,19,19.8,2.5,4.9,.512,...,1.7,3.8,5.5,1.5,0.5,0.6,1.0,2.0,6.9,2018
4,Arron Afflalo,SG,32,ORL,53,3,12.9,1.2,3.1,.401,...,0.1,1.2,1.2,0.6,0.1,0.2,0.4,1.1,3.4,2018


In [50]:
#Renaming so the year column is named appropriately
df = df.rename(columns={'Rk' : 'Year'})

In [52]:
#Ensuring the year column is as int instead of float
df['Year'] = df['Year'].astype(int)